In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing 
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

2024-08-14 15:16:21.303802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 15:16:21.303920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 15:16:21.439529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

/tmp/ipykernel_34/2653809078.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [3]:
df_2018 = pd.read_csv('/kaggle/input/isic-2018-jpg-224x224-resized/train-metadata.csv')
df_2019 = pd.read_csv('/kaggle/input/isic-2019-jpg-224x224-resized/train-metadata.csv')
df_2020 = pd.read_csv('/kaggle/input/isic-2020-jpg-224x224-resized/train-metadata.csv')

In [4]:
# df_2019.head()

## Constants

In [5]:
RANDOM_STATE = 42
IMAGE_SIZE=224
BATCH_SIZE = 64

# img_augmentation_layers = [
#     layers.RandomRotation(factor=0.15),
#     layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
#     layers.RandomFlip(),
#     layers.RandomContrast(factor=0.1),
# ]




## Helper functions

In [6]:
def get_sample(df,ratio):
    df_positive = df[df['target']==1].reset_index(drop=True)
    df_negative = df[df['target']==0].reset_index(drop=True)
    df_negative= resample(df_negative,n_samples=len(df_positive)*ratio,random_state=RANDOM_STATE)
#     df_positive = resample(df_positive,n_samples=len(df_positive),random_state=RANDOM_STATE)
    
    df_concat = pd.concat([df_positive,df_negative])
    df_concat = shuffle(df_concat).reset_index(drop=True)
    
    print(f'positive shape: {df_positive.shape}, negative shape: {df_negative.shape}')
    print(f'Full dataset shape: {df_concat.shape}')
    
    return df_concat

def visualize(df,idx):
    images = []
    labels = []
    for id in idx:
        current_row = df.iloc[id]
        images.append(plt.imread(current_row["file_path"]))
        labels.append(current_row["target"])
    
    for i, (image, label) in enumerate(zip(images, labels), start=1):
        plt.subplot(3, 3, i)  # Adjusted to start from 1
        plt.imshow(image)
        plt.axis("off")
        plt.title(label)
    plt.show()

In [7]:
def calculate_weight(pos,neg):

    # Total number of samples
    total = pos + neg

    # Calculate class weights
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0 (Negative): {:.2f}'.format(weight_for_0))
    print('Weight for class 1 (Positive): {:.2f}'.format(weight_for_1))
    return class_weight

In [8]:
import gc
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

clear_memory = ClearMemory()


In [9]:
class PartialAUC(tf.keras.metrics.Metric):
    def __init__(self, name='partial_auc', **kwargs):
        super(PartialAUC, self).__init__(name=name, **kwargs)
        self.thresholds = tf.linspace(0.0, 1.0, 101)
        self.tp = self.add_weight(name='tp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fp = self.add_weight(name='fp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fn = self.add_weight(name='fn', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.tn = self.add_weight(name='tn', shape=(101,), initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(tf.cast(y_true, tf.float32), (-1, 1))
        y_pred = tf.cast(y_pred, tf.float32)

        # Debugging shapes
        #tf.print("reshaped y_true shape:", tf.shape(y_true))
        #tf.print("y_pred shape:", tf.shape(y_pred))

        thresholds = self.thresholds[tf.newaxis, :]
        preds = tf.cast(y_pred[:, tf.newaxis] >= thresholds, tf.float32)

        tp = tf.reduce_sum(preds * y_true[:, tf.newaxis], axis=0)
        fp = tf.reduce_sum(preds * (1 - y_true)[:, tf.newaxis], axis=0)
        fn = tf.reduce_sum((1 - preds) * y_true[:, tf.newaxis], axis=0)
        tn = tf.reduce_sum((1 - preds) * (1 - y_true)[:, tf.newaxis], axis=0)

        # Check tensor shapes
        #tf.print("tp shape:", tf.shape(tp))
        #tf.print("fp shape:", tf.shape(fp))
        #tf.print("fn shape:", tf.shape(fn))
        #tf.print("tn shape:", tf.shape(tn))

        # Reshape assuming desired shape is (num_thresholds,)
        num_thresholds = tf.shape(self.thresholds)[0]
        
        reshaped_tp = tf.reshape(tp, [num_thresholds])
        reshaped_fp = tf.reshape(fp, [num_thresholds])
        reshaped_fn = tf.reshape(fn, [num_thresholds])
        reshaped_tn = tf.reshape(tn, [num_thresholds])
        
        self.tp.assign_add(reshaped_tp)
        self.fp.assign_add(reshaped_fp)
        self.fn.assign_add(reshaped_fn)
        self.tn.assign_add(reshaped_tn)

    def result(self):
        # Avoid division by zero
        epsilon = tf.keras.backend.epsilon()
        tpr = self.tp / (self.tp + self.fn + epsilon)
        fpr = self.fp / (self.fp + self.tn + epsilon)

        # Sort fpr and corresponding tpr
        sorted_indices = tf.argsort(fpr)
        sorted_fpr = tf.gather(fpr, sorted_indices)
        sorted_tpr = tf.gather(tpr, sorted_indices)

        # Apply mask for TPR >= 0.8
        mask = sorted_tpr >= 0.8
        partial_fpr = tf.boolean_mask(sorted_fpr, mask)
        partial_tpr = tf.boolean_mask(sorted_tpr, mask)

        # Calculate partial AUC using the trapezoidal rule
        def compute_partial_auc():
            delta_fpr = partial_fpr[1:] - partial_fpr[:-1]
            tpr_sum = partial_tpr[1:] + partial_tpr[:-1]
            return tf.reduce_sum(delta_fpr * tpr_sum / 2)

        partial_auc = tf.cond(tf.shape(partial_fpr)[0] > 1,
                              compute_partial_auc,
                              lambda: tf.constant(0.0))

        return partial_auc * 0.2

    def reset_states(self):
        self.tp.assign(tf.zeros_like(self.tp))
        self.fp.assign(tf.zeros_like(self.fp))
        self.fn.assign(tf.zeros_like(self.fn))
        self.tn.assign(tf.zeros_like(self.tn))

In [10]:
df_2018 = get_sample(df_2018,ratio=10)
df_2018['target'] = df_2018['target'].astype(str)
df_2018['isic_id'] = df_2018['isic_id'] + ".jpg" 

positive shape: (785, 4), negative shape: (7850, 4)
Full dataset shape: (8635, 4)


In [11]:
train_datagen_2018 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2
)

train_2018 = train_datagen_2018.flow_from_dataframe(df_2018,
                                              directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
                                              x_col="isic_id",
                                              y_col="target",
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              batch_size=BATCH_SIZE,
                                              class_mode='binary',
                                              )
val_2018 = train_datagen_2018.flow_from_dataframe(
    df_2018,
    directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 7205 validated image filenames belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1430 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(


Found 1441 validated image filenames belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1430 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(


In [12]:
df_2019 = get_sample(df_2019,ratio=10)
df_2019['target'] = df_2019['target'].astype(str)
df_2019['isic_id'] = df_2019['isic_id'] + ".jpg" 
train_datagen_2019 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)
# Training set
train_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

val_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

positive shape: (4522, 4), negative shape: (45220, 4)
Full dataset shape: (49742, 4)
Found 39794 validated image filenames belonging to 2 classes.
Found 9948 validated image filenames belonging to 2 classes.


In [13]:
df_2020 = get_sample(df_2020,ratio=10)

positive shape: (584, 4), negative shape: (5840, 4)
Full dataset shape: (6424, 4)


In [14]:
train_datagen_2020 = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)
df_2020['target'] = df_2020['target'].astype(str)
df_2020['isic_id'] = df_2020['isic_id'] + ".jpg" \

train_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

# Validation set
val_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 5140 validated image filenames belonging to 2 classes.
Found 1284 validated image filenames belonging to 2 classes.


In [15]:
df = get_sample(df,ratio=20)
df['target'] = df['target'].astype(str)
df['isic_id'] = df['isic_id'] + ".jpg" 
train_datagen = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    channel_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)

train_data = train_datagen.flow_from_dataframe(
    df,
    directory="/kaggle/input/isic-2024-challenge/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training' 
)

positive shape: (393, 55), negative shape: (7860, 55)
Full dataset shape: (8253, 55)
Found 6603 validated image filenames belonging to 2 classes.


## Vision Model

In [16]:
from keras.applications import EfficientNetB0

In [17]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="/kaggle/input/efficient-net-b0-no-top-weights/efficientnetb0_notop.h5")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=[PartialAUC()]
    )
    return model
# Use the strategy to scope model creation and training
# with strategy.scope():
#     eff_model = EfficientNetB0(
#     include_top=True,
#     weights='/kaggle/input/efficientnet-keras-weights-b0b5/efficientnet-b0_imagenet_1000_notop.h5',
#     classes=2,
#     input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
#     )
# eff_model.load_weights('/kaggle/input/efficientnet-keras-weights-b0b5/efficientnet-b0_imagenet_1000_notop.h5')
# eff_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[PartialAUC()])

# eff_model.summary()

# epochs = 5  # @param {type: "slider", min:10, max:100}
# hist = eff_model.fit(train_data, epochs=epochs)

In [18]:
strategy = tf.distribute.MirroredStrategy()

# Use the strategy to scope model creation and training
with strategy.scope():
    eff_model = build_model(num_classes=1)

In [19]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2018 = eff_model.fit(train_2018,validation_data=val_2018, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5


2024-08-14 15:21:37.491171: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_729/cond/StatefulPartitionedCall/EfficientNet_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


113/113 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 0.4346 - partial_auc: 0.0292 - val_loss: 0.3010 - val_partial_auc: 0.0000e+00
Epoch 2/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - loss: 0.3783 - partial_auc: 0.0259 - val_loss: 0.3093 - val_partial_auc: 0.0000e+00
Epoch 3/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - loss: 0.3466 - partial_auc: 0.0330 - val_loss: 0.2959 - val_partial_auc: 0.0000e+00
Epoch 4/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - loss: 0.3413 - partial_auc: 0.0337 - val_loss: 0.3130 - val_partial_auc: 0.0000e+00
Epoch 5/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - loss: 0.3375 - partial_auc: 0.0344 - val_loss: 0.2638 - val_partial_auc: 0.0000e+00


In [21]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2019 = eff_model.fit(train_2019,validation_data=val_2019, epochs=2,callbacks=[clear_memory])

Epoch 1/2
622/622 ━━━━━━━━━━━━━━━━━━━━ 716s 1s/step - loss: 0.3148 - partial_auc: 0.0297 - val_loss: 0.3039 - val_partial_auc: 0.0000e+00
Epoch 2/2
622/622 ━━━━━━━━━━━━━━━━━━━━ 714s 1s/step - loss: 0.3110 - partial_auc: 0.0269 - val_loss: 0.3045 - val_partial_auc: 0.0000e+00


In [22]:
epochs = 5  # @param {type: "slider", min:8, max:80}
hist_2020 = eff_model.fit(train_2020,validation_data=val_2020, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - loss: 0.3138 - partial_auc: 0.0241 - val_loss: 0.2822 - val_partial_auc: 0.0000e+00
Epoch 2/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - loss: 0.3027 - partial_auc: 0.0220 - val_loss: 0.3210 - val_partial_auc: 0.0000e+00
Epoch 3/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 0.3079 - partial_auc: 0.0172 - val_loss: 0.3148 - val_partial_auc: 0.0000e+00
Epoch 4/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - loss: 0.3113 - partial_auc: 0.0266 - val_loss: 0.2997 - val_partial_auc: 0.0000e+00
Epoch 5/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - loss: 0.3207 - partial_auc: 0.0204 - val_loss: 0.2821 - val_partial_auc: 0.0000e+00


In [23]:
hist_2020 = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory])

Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - loss: 0.1938 - partial_auc: 0.0197
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 106s 955ms/step - loss: 0.2029 - partial_auc: 0.0250
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 106s 951ms/step - loss: 0.1943 - partial_auc: 0.0190
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 106s 947ms/step - loss: 0.2034 - partial_auc: 0.0195
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 105s 941ms/step - loss: 0.2015 - partial_auc: 0.0184


In [24]:
len(eff_model.layers)

241

In [42]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-80:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=[PartialAUC()]
    )
    return model

with strategy.scope():
    eff_model = unfreeze_model(eff_model)

In [43]:
eff_model.summary()

Model: "EfficientNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_

 Total params: 4,055,972 (15.47 MB)

 Trainable params: 3,133,725 (11.95 MB)

 Non-trainable params: 922,247 (3.52 MB)

In [44]:
hist_2018 = eff_model.fit(train_2018,validation_data=val_2018, epochs=epochs,callbacks=[clear_memory])

Epoch 1/4


2024-08-14 16:51:14.354517: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_873/cond/StatefulPartitionedCall/EfficientNet_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


113/113 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - loss: 0.3243 - partial_auc: 0.0258 - val_loss: 0.2900 - val_partial_auc: 0.0000e+00
Epoch 2/4
113/113 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - loss: 0.3253 - partial_auc: 0.0218 - val_loss: 0.3567 - val_partial_auc: 0.0000e+00
Epoch 3/4
113/113 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - loss: 0.3228 - partial_auc: 0.0291 - val_loss: 0.3290 - val_partial_auc: 0.0000e+00
Epoch 4/4
113/113 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - loss: 0.3169 - partial_auc: 0.0253 - val_loss: 0.3065 - val_partial_auc: 0.0000e+00


In [ ]:
hist_2020 = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory])

In [37]:
epochs = 4  # @param {type: "slider", min:4, max:10}
hist = eff_model.fit(train_data, epochs=epochs,callbacks=[clear_memory])

Epoch 1/4


KeyboardInterrupt: 

In [ ]:
strategy = tf.distribute.MirroredStrategy()

# Use the strategy to scope model creation and training
with strategy.scope():
    model = vision_model()

In [70]:
def create_vision_encoder(trainable_layer, trainable=False, input_shape=(139,139,3)):
    # Load ResNet50V2 model with lower computational complexity
    model = keras.applications.ResNet50V2(include_top=False, weights='/kaggle/input/resnet-weights/resnet50v2_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=input_shape)
    
    # Freeze all layers by default
    model.trainable = False
    
    # Optionally unfreeze layers up to trainable_layer
    if trainable:
        for layer in model.layers[-10:]:
            layer.trainable = trainable
    
    # Define input and apply preprocessing
    image = keras.Input(shape=input_shape, name="image")
    preprocessed = keras.applications.resnet_v2.preprocess_input(image)
    
    # Get the output from the model
    outputs = model(preprocessed)
    
    # Return the final model
    return keras.Model(inputs=[image], outputs=outputs, name="vision_encoder")


In [77]:
def vision_model():
    vision_encoder = create_vision_encoder(trainable_layer=235,input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),trainable=True)
    
    x = layers.MaxPooling2D()(vision_encoder.output)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64,activation='relu')(x)
    x = layers.Dense(1,activation='sigmoid')(x)

    vision_model = keras.Model(inputs = vision_encoder.input, outputs=x)
    vision_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[PartialAUC()])
    return vision_model

In [78]:
strategy = tf.distribute.MirroredStrategy()

# Use the strategy to scope model creation and training
with strategy.scope():
    model = vision_model()

In [79]:
model

<Functional name=functional_1, built=True>

In [88]:
history = model.fit(train_data, epochs=10)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


258/258 ━━━━━━━━━━━━━━━━━━━━ 70s 150ms/step - loss: 0.4052 - partial_auc: 0.0088
Epoch 2/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.2111 - partial_auc: 0.0255
Epoch 3/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1986 - partial_auc: 0.0339
Epoch 4/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1927 - partial_auc: 0.0442
Epoch 5/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1651 - partial_auc: 0.0579
Epoch 6/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1820 - partial_auc: 0.0532
Epoch 7/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - loss: 0.1895 - partial_auc: 0.0498
Epoch 8/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1675 - partial_auc: 0.0679
Epoch 9/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - loss: 0.1755 - partial_auc: 0.0743
Epoch 10/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - loss: 0.1869 - partial_auc: 0.0697


In [89]:
import h5py


In [90]:
hdf5_file_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'

In [91]:
hdf5_file = h5py.File(hdf5_file_path, 'r')

def preprocess_image(image_bytes):
    image = tf.io.decode_jpeg(image_bytes)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

image_list = []
image_ids = []

for key in hdf5_file.keys():
    image_bytes = hdf5_file[key][()]
    image = preprocess_image(image_bytes)
    
    image_list.append(image)
    image_ids.append(key)
images_tensor = tf.stack(image_list)

# You now have the images ready for prediction
# Example: predictions = model.predict(images_tensor)

# Make sure to close the file
hdf5_file.close()

In [99]:
csv = pd.read_csv('/kaggle/input/isic-2024-challenge/sample_submission.csv')
csv.head()


,isic_id,target
0,ISIC_0015657,0
1,ISIC_0015729,0
2,ISIC_0015740,0


https://github.com/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment.ipynb

In [103]:
my = pd.read_csv('/kaggle/working/submission.csv')
my.head()

,isic_id,target
0,ISIC_0015657,0
1,ISIC_0015729,0
2,ISIC_0015740,0


In [102]:
predictions = model.predict(images_tensor)
predicted_classes = np.argmax(predictions, axis=1)
submission = pd.DataFrame({
    'isic_id': image_ids,
    'target': predicted_classes
})

submission.to_csv('submission.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
